In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import zipfile
import shutil
import random
import pandas as pd
import csv
import os

In [3]:
# Paths to training and testing data
train_data_path = '/Users/raghavgarg/Downloads/archive (1)/seg_train/seg_train/'
test_data_path = '/Users/raghavgarg/Downloads/archive (1)/seg_test/seg_test/'

# Output directories for models and reports
output_dir = '/Users/raghavgarg/Downloads'
models_dir = os.path.join(output_dir, "models")
reports_dir = os.path.join(output_dir, "reports")
os.makedirs(models_dir, exist_ok=True)
os.makedirs(reports_dir, exist_ok=True)
classes = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [2]:
import os

def get_dataset_stats(directory):
    class_counts = {}
    total_images = 0  # Initialize total image counter

    # Loop through each class folder in the directory
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):  # Check if it's a folder
            image_count = len(os.listdir(class_path))  # Count files in the folder
            class_counts[class_name] = image_count
            total_images += image_count  # Add to the total count

    # Get the class with min and max images
    min_class = min(class_counts, key=class_counts.get)
    max_class = max(class_counts, key=class_counts.get)

    return {
        "min": (min_class, class_counts[min_class]),
        "max": (max_class, class_counts[max_class]),
        "total": total_images
    }

# Paths to train and validation directories
train_dir = '/Users/raghavgarg/Downloads/archive (1)/seg_train/seg_train/'
validation_dir = '/Users/raghavgarg/Downloads/archive (1)/seg_test/seg_test/'

# Get stats for training and validation datasets
train_stats = get_dataset_stats(train_dir)
validation_stats = get_dataset_stats(validation_dir)

# Print results
print("Training Dataset:")
print(f"Class with minimum images: {train_stats['min'][0]} ({train_stats['min'][1]} images)")
print(f"Class with maximum images: {train_stats['max'][0]} ({train_stats['max'][1]} images)")
print(f"Total images: {train_stats['total']}")

print("\nValidation Dataset:")
print(f"Class with minimum images: {validation_stats['min'][0]} ({validation_stats['min'][1]} images)")
print(f"Class with maximum images: {validation_stats['max'][0]} ({validation_stats['max'][1]} images)")
print(f"Total images: {validation_stats['total']}")


Training Dataset:
Class with minimum images: buildings (2191 images)
Class with maximum images: mountain (2512 images)
Total images: 14034

Validation Dataset:
Class with minimum images: buildings (437 images)
Class with maximum images: glacier (553 images)
Total images: 3000


In [4]:
#Set data augmentation techniques
train_datagen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,vertical_flip=True
                                                             ,zoom_range=0.2,rotation_range=360
                                                             ,width_shift_range=0.1,height_shift_range=0.1
                                                             ,channel_shift_range=50
                                                             ,brightness_range=(0,1.2)
                                                             ,preprocessing_function=keras.applications.imagenet_utils.preprocess_input)

test_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.imagenet_utils.preprocess_input)

In [5]:
# Loading images directly from directories
batch_size = 70
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)
validation_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [7]:
import keras
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import csv
import json
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K

# Helper function to calculate F1 Score, Sensitivity, and Specificity
def calculate_metrics(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    precisions, recalls, f1_scores, specificities = [], [], [], []

    for i in range(cm.shape[0]):
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp
        fp = cm[:, i].sum() - tp
        tn = cm.sum() - (tp + fn + fp)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1_score)
        specificities.append(specificity)

    return {
        'f1_score': np.mean(f1_scores),
        'sensitivity': np.mean(recalls),
        'specificity': np.mean(specificities)
    }

# Custom callback to save metrics and checkpoints
class SaveMetricsAndCheckpoints(keras.callbacks.Callback):
    def __init__(self, validation_data, log_file_path='/kaggle/working/metrics_log.csv', save_interval=10):
        super().__init__()
        self.validation_data = validation_data
        self.log_file_path = log_file_path
        self.save_interval = save_interval

        # Initialize the CSV file with headers
        with open(self.log_file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                'epoch',
                'train_loss', 'train_accuracy', 'train_precision', 'train_recall', 
                'train_top_1_accuracy', 'train_top_5_accuracy',
                'val_loss', 'val_accuracy', 'val_precision', 'val_recall',
                'val_top_1_accuracy', 'val_top_5_accuracy',
                'f1_score', 'sensitivity', 'specificity'
            ])
        print(f"Metrics will be logged to: {self.log_file_path}")

    def on_epoch_end(self, epoch, logs=None):
        # Extract validation data
        val_images, val_labels = self.validation_data
        y_pred = self.model.predict(val_images)
        y_pred_labels = tf.argmax(y_pred, axis=1).numpy()
        y_true_labels = tf.argmax(val_labels, axis=1).numpy()

        # Calculate additional metrics
        additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

        # Save metrics to the CSV file
        with open(self.log_file_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                epoch + 1,
                logs.get('loss'), logs.get('accuracy'), logs.get('precision'), logs.get('recall'),
                logs.get('top_1_accuracy'), logs.get('top_5_accuracy'),
                logs.get('val_loss'), logs.get('val_accuracy'), logs.get('val_precision'), logs.get('val_recall'),
                logs.get('val_top_1_accuracy'), logs.get('val_top_5_accuracy'),
                additional_metrics['f1_score'], additional_metrics['sensitivity'], additional_metrics['specificity']
            ])
        
        print(f"Epoch {epoch + 1}: Metrics logged.")

        # Save model checkpoint every nth epoch
        if (epoch + 1) % self.save_interval == 0:
            checkpoint_filepath = (
                f"/kaggle/working/model-"
                f"{epoch + 1:02d}-"
                f"val_acc_{logs['val_accuracy']:.4f}.keras"
            )
            self.model.save(checkpoint_filepath)
            print(f"Model checkpoint saved at: {checkpoint_filepath}")

# Validation data
validation_data = next(iter(validation_generator))

# Initialize callback
metrics_and_checkpoint_callback = SaveMetricsAndCheckpoints(
    validation_data=(validation_data[0], validation_data[1]),
    log_file_path='/Users/raghavgarg/Downloads/metrics_log.csv',
    save_interval=10
)


Metrics will be logged to: /Users/raghavgarg/Downloads/metrics_log.csv


## DENSE_AVG_DEPTH_CONST_224

In [28]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/DENSENET169/AVG_DEPTH_224/DENSE_AVG_DEPTH_CONST_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/DENSENET169/AVG_DEPTH_224/DENSE_AVG_DEPTH_CONST_224.h5


In [29]:
model.count_params()


12669510

In [10]:
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.8705 - loss: 0.4487 - precision: 0.8774 - recall: 0.8662 - top_1_accuracy: 0.8705 - top_5_accuracy: 0.9986
Test Loss: 0.4567815065383911
Test Accuracy: 0.8733333349227905
Test Precision: 0.8797703385353088
Test Recall: 0.8683333396911621
Top-1 Accuracy: 0.8733333349227905
Top-5 Accuracy: 0.9986666440963745
F1 Score: 0.8734334733158223
Sensitivity (Recall): 0.8743028322880056
Specificity: 0.9746166329444694


## DENSE_DEPTH_CONST_224

In [30]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/DENSENET169/DEPTH_224/dense_depth_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

12736070

In [11]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/DENSENET169/DEPTH_224/dense_depth_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/DENSENET169/DEPTH_224/dense_depth_224.h5
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 103s 2s/step - accuracy: 0.8788 - loss: 0.4579 - precision: 0.8816 - recall: 0.8755 - top_1_accuracy: 0.8788 - top_5_accuracy: 0.9998
Test Loss: 0.47070300579071045
Test Accuracy: 0.8773333430290222
Test Precision: 0.8815213441848755
Test Recall: 0.8730000257492065
Top-1 Accuracy: 0.8773333430290222
Top-5 Accuracy: 0.9993333220481873
F1 Score: 0.8777040645410156
Sensitivity (Recall): 0.8785256702681083
Specificity: 0.975419281599922


## DENSE GAP 224

In [31]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/DENSENET169/GAP 224/DENSENET224GAP.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

12652870

In [12]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/DENSENET169/GAP 224/DENSENET224GAP.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/DENSENET169/GAP 224/DENSENET224GAP.h5
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 104s 2s/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.8310 - loss: 0.6586 - precision: 0.8505 - recall: 0.8228 - top_1_accuracy: 0.8310 - top_5_accuracy: 0.9986
Test Loss: 0.5717562437057495
Test Accuracy: 0.8450000286102295
Test Precision: 0.8619979619979858
Test Recall: 0.8370000123977661
Top-1 Accuracy: 0.8450000286102295
Top-5 Accuracy: 0.9980000257492065
F1 Score: 0.8449678160517546
Sensitivity (Recall): 0.8453046843125006
Specificity: 0.9689796285968605


## EFFICIENT AVG DEPTH 224

In [32]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/EFFICIENTNETB0/AVG_DEPTH_224/effi_avg_depth_224.keras'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

4070057

In [13]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/EFFICIENTNETB0/AVG_DEPTH_224/effi_avg_depth_224.keras'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/EFFICIENTNETB0/AVG_DEPTH_224/effi_avg_depth_224.keras
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1734685024.939443 1938576 service.cc:148] XLA service 0x600003d9cb00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734685024.939953 1938576 service.cc:156]   StreamExecutor device (0): Host, Default Version
2024-12-20 14:27:05.036927: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1734685025.681473 1938576 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 ━━━━━━━━━━━━━━━━━━━━ 215s 5s/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 214s 5s/step - accuracy: 0.8689 - loss: 0.3671 - precision: 0.8793 - recall: 0.8635 - top_1_accuracy: 0.8689 - top_5_accuracy: 0.9998
Test Loss: 0.3455387353897095
Test Accuracy: 0.8830000162124634
Test Precision: 0.8911794424057007
Test Recall: 0.8790000081062317
Top-1 Accuracy: 0.8830000162124634
Top-5 Accuracy: 0.999666690826416
F1 Score: 0.8830475749804169
Sensitivity (Recall): 0.8839436870538849
Specificity: 0.9765564883237273


## EFFICIENT DEPTH 224

In [33]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/EFFICIENTNETB0/DEPTH_224/effi_dense_224.keras'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

4121257

In [15]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/EFFICIENTNETB0/DEPTH_224/effi_dense_224.keras'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/EFFICIENTNETB0/DEPTH_224/effi_dense_224.keras
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 202s 5s/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 208s 5s/step - accuracy: 0.8837 - loss: 0.3544 - precision: 0.8886 - recall: 0.8783 - top_1_accuracy: 0.8837 - top_5_accuracy: 0.9996
Test Loss: 0.34710028767585754
Test Accuracy: 0.887333333492279
Test Precision: 0.8941494822502136
Test Recall: 0.8813333511352539
Top-1 Accuracy: 0.887333333492279
Top-5 Accuracy: 0.9993333220481873
F1 Score: 0.888180867336754
Sensitivity (Recall): 0.888961393579772
Specificity: 0.9773959066233836


## EFFICIENT GAP 224

In [34]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/EFFICIENTNETB0/GAP_224/effi_gap_224.keras'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

4057257

In [18]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/EFFICIENTNETB0/GAP_224/effi_gap_224.keras'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/EFFICIENTNETB0/GAP_224/effi_gap_224.keras
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 220s 5s/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 207s 5s/step - accuracy: 0.8683 - loss: 0.3736 - precision: 0.8760 - recall: 0.8602 - top_1_accuracy: 0.8683 - top_5_accuracy: 0.9995
Test Loss: 0.35532158613204956
Test Accuracy: 0.8790000081062317
Test Precision: 0.885840117931366
Test Recall: 0.871666669845581
Top-1 Accuracy: 0.8790000081062317
Top-5 Accuracy: 0.9993333220481873
F1 Score: 0.8792846858421606
Sensitivity (Recall): 0.8801301148403716
Specificity: 0.9757383716917203


## MOBILENETV2 AVG DENSE 224

In [35]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/MOBILENETV2/AVG_DEPTH_224/mobile_avg_depth_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

2278470

In [19]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/MOBILENETV2/AVG_DEPTH_224/mobile_avg_depth_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/MOBILENETV2/AVG_DEPTH_224/mobile_avg_depth_224.h5
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 21s 468ms/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 20s 453ms/step - accuracy: 0.8746 - loss: 0.3589 - precision: 0.8786 - recall: 0.8711 - top_1_accuracy: 0.8746 - top_5_accuracy: 0.9998
Test Loss: 0.36707186698913574
Test Accuracy: 0.8823333382606506
Test Precision: 0.8883265852928162
Test Recall: 0.8776666522026062
Top-1 Accuracy: 0.8823333382606506
Top-5 Accuracy: 0.999666690826416
F1 Score: 0.8827716040581975
Sensitivity (Recall): 0.8836673709481602
Specificity: 0.9763854837114273


## MOBILE DENSE 224

In [36]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/MOBILENETV2/DEPTH_224/mobile_depth_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

2329670

In [20]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/MOBILENETV2/DEPTH_224/mobile_depth_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/MOBILENETV2/DEPTH_224/mobile_depth_224.h5
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 20s 450ms/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 20s 445ms/step - accuracy: 0.8982 - loss: 0.3089 - precision: 0.9049 - recall: 0.8943 - top_1_accuracy: 0.8982 - top_5_accuracy: 0.9998
Test Loss: 0.3144559860229492
Test Accuracy: 0.8999999761581421
Test Precision: 0.9068511724472046
Test Recall: 0.8956666588783264
Top-1 Accuracy: 0.8999999761581421
Top-5 Accuracy: 0.999666690826416
F1 Score: 0.9006155259312997
Sensitivity (Recall): 0.9014277321834894
Specificity: 0.9799509206684859


## MOBILE GAP 224

In [37]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/MOBILENETV2/GAP_224/MobileNetV2_GAP_220.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

2265670

In [21]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/MOBILENETV2/GAP_224/MobileNetV2_GAP_220.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/MOBILENETV2/GAP_224/MobileNetV2_GAP_220.h5
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 21s 478ms/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 20s 456ms/step - accuracy: 0.8801 - loss: 0.3804 - precision: 0.8865 - recall: 0.8772 - top_1_accuracy: 0.8801 - top_5_accuracy: 0.9993
Test Loss: 0.3723303973674774
Test Accuracy: 0.8886666893959045
Test Precision: 0.8954821228981018
Test Recall: 0.8853333592414856
Top-1 Accuracy: 0.8886666893959045
Top-5 Accuracy: 0.9986666440963745
F1 Score: 0.8888566293414365
Sensitivity (Recall): 0.8897412509854107
Specificity: 0.9776948314471542


## RESNET AVG DEPTH 224

In [38]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/RESNET50/AVG_DEPTH_224/resnet_avg_depth_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

23620486

In [22]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/RESNET50/AVG_DEPTH_224/resnet_avg_depth_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/RESNET50/AVG_DEPTH_224/resnet_avg_depth_224.h5
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 87s 2s/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 87s 2s/step - accuracy: 0.8676 - loss: 0.4520 - precision: 0.8739 - recall: 0.8640 - top_1_accuracy: 0.8676 - top_5_accuracy: 0.9994
Test Loss: 0.4346264600753784
Test Accuracy: 0.8730000257492065
Test Precision: 0.8785834908485413
Test Recall: 0.8683333396911621
Top-1 Accuracy: 0.8730000257492065
Top-5 Accuracy: 0.9990000128746033
F1 Score: 0.8729978936652358
Sensitivity (Recall): 0.8738152274895196
Specificity: 0.9745747462377571


## RESNET DEPTH 224

In [39]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/RESNET50/DEPTH_224/resnet_depthwise_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

23702406

In [23]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/RESNET50/DEPTH_224/resnet_depthwise_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/RESNET50/DEPTH_224/resnet_depthwise_224.h5
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 89s 2s/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 89s 2s/step - accuracy: 0.8658 - loss: 0.4924 - precision: 0.8739 - recall: 0.8594 - top_1_accuracy: 0.8658 - top_5_accuracy: 0.9988
Test Loss: 0.5101150870323181
Test Accuracy: 0.856333315372467
Test Precision: 0.8629441857337952
Test Recall: 0.8500000238418579
Top-1 Accuracy: 0.856333315372467
Top-5 Accuracy: 0.9980000257492065
F1 Score: 0.8563241874225612
Sensitivity (Recall): 0.8576067608985362
Specificity: 0.9712597361816885


## RESNET GAP 224

In [40]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/RESNET50/GAP_224/RESNET50_GAP_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

23600006

In [24]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/RESNET50/GAP_224/RESNET50_GAP_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/RESNET50/GAP_224/RESNET50_GAP_224.h5
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.8326 - loss: 0.5738 - precision: 0.8421 - recall: 0.8226 - top_1_accuracy: 0.8326 - top_5_accuracy: 0.9986
Test Loss: 0.6232506632804871
Test Accuracy: 0.8489999771118164
Test Precision: 0.8608636260032654
Test Recall: 0.8373333215713501
Top-1 Accuracy: 0.8489999771118164
Top-5 Accuracy: 0.9973333477973938
F1 Score: 0.8483424102065027
Sensitivity (Recall): 0.8492219209714795
Specificity: 0.9697404433853292


## XCEPTION AVG DENSE 224

In [41]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/XCEPTIONNET/AVG_DEPTH_224/xcep_avg_depth_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

20894254

In [25]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/XCEPTIONNET/AVG_DEPTH_224/xcep_avg_depth_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/XCEPTIONNET/AVG_DEPTH_224/xcep_avg_depth_224.h5
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 114s 3s/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 126s 3s/step - accuracy: 0.9012 - loss: 0.4541 - precision: 0.9054 - recall: 0.9005 - top_1_accuracy: 0.9012 - top_5_accuracy: 0.9991
Test Loss: 0.4949043393135071
Test Accuracy: 0.8939999938011169
Test Precision: 0.8959866166114807
Test Recall: 0.8930000066757202
Top-1 Accuracy: 0.8939999938011169
Top-5 Accuracy: 0.9986666440963745
F1 Score: 0.8949080702977912
Sensitivity (Recall): 0.8958142865918859
Specificity: 0.9787375796839252


## XCEPTION DENSE 224 

In [42]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/XCEPTIONNET/DEPTH_224/XCEP_DEPTH_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

20976174

In [26]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/XCEPTIONNET/DEPTH_224/XCEP_DEPTH_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/XCEPTIONNET/DEPTH_224/XCEP_DEPTH_224.h5
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 112s 3s/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 105s 2s/step - accuracy: 0.8932 - loss: 0.5009 - precision: 0.8953 - recall: 0.8930 - top_1_accuracy: 0.8932 - top_5_accuracy: 0.9992
Test Loss: 0.49575650691986084
Test Accuracy: 0.893666684627533
Test Precision: 0.8957219123840332
Test Recall: 0.8933333158493042
Top-1 Accuracy: 0.893666684627533
Top-5 Accuracy: 0.9990000128746033
F1 Score: 0.8944645386044646
Sensitivity (Recall): 0.8948481026623324
Specificity: 0.9786765892262567


## XCEPTION GAP 224

In [43]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/XCEPTIONNET/GAP_224/Xcep_GAP_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)
model.count_params()

20873774

In [27]:
# Load the model from the saved checkpoint
model_path = '/Users/raghavgarg/Downloads/RESEARCH/XCEPTIONNET/GAP_224/Xcep_GAP_224.h5'  # Update the model path as needed
model = keras.models.load_model(model_path)

print(f"Model loaded from: {model_path}")
# Test Data Evaluation
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Ground truth labels from the test generator
y_true_labels = test_generator.classes

# Calculate additional metrics like precision, recall, F1 score, etc.
additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

# Evaluate the model on test data to get Top-1 and Top-5 accuracy
test_loss, test_accuracy, test_precision, test_recall, top_1_acc, top_5_acc = model.evaluate(test_generator)

# Print the metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Top-1 Accuracy: {top_1_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")
print(f"F1 Score: {additional_metrics['f1_score']}")
print(f"Sensitivity (Recall): {additional_metrics['sensitivity']}")
print(f"Specificity: {additional_metrics['specificity']}")


Model loaded from: /Users/raghavgarg/Downloads/RESEARCH/XCEPTIONNET/GAP_224/Xcep_GAP_224.h5
Found 3000 images belonging to 6 classes.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


43/43 ━━━━━━━━━━━━━━━━━━━━ 105s 2s/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 111s 3s/step - accuracy: 0.8778 - loss: 0.5826 - precision: 0.8789 - recall: 0.8750 - top_1_accuracy: 0.8778 - top_5_accuracy: 0.9991
Test Loss: 0.6819871664047241
Test Accuracy: 0.862666666507721
Test Precision: 0.8649916052818298
Test Recall: 0.8606666922569275
Top-1 Accuracy: 0.862666666507721
Top-5 Accuracy: 0.9990000128746033
F1 Score: 0.8630366849190936
Sensitivity (Recall): 0.8650028955305196
Specificity: 0.9725696263140943
